In [1]:
import numpy as np
import random

In [11]:
# Sparse signal 
n, ps, rhos = 100, [10, 25, 50], [0, 0.25, 0.5]

In [17]:
def generate_data(n, p, rho, signal='sparse'):
    if signal == 'sparse':
        beta = np.array([[2/np.sqrt(n) * (j < np.sqrt(p)) for j in range(1,p+1,1)]]).T
    elif signal == 'dense':
        beta = np.array([[5/(j*np.sqrt(n)) for j in range(1,p+1,1)]]).T
    else:
        return
    R2 = 0.8
    sigma_rho = np.reshape([rho**(np.abs(i-j)) for i in range(1,p+1,1) for j in range(1,p+1,1)], (-1, p))
    sigma2 = ((1-R2)/R2 * (beta.T) @ sigma_rho @ beta)[0][0]
    sigma = np.sqrt(sigma2)
    epsilon = np.random.normal(0, sigma, n).reshape(n,1)
    X = np.random.multivariate_normal(np.zeros(p), sigma_rho, n)
    Y = X @ beta + epsilon
    return X, Y

In [ ]:
def generate_data_dense(n, p, rho):
    beta = np.array([[2/np.sqrt(n) * (j < np.sqrt(p)) for j in range(p)]]).T
    R2 = 0.8
    sigma_rho = np.reshape([rho**(np.abs(i-j)) for i in range(p) for j in range(p)], (-1, p))
    sigma2 = ((1-R2)/R2 * (beta.T) @ sigma_rho @ beta)[0][0]
    sigma = np.sqrt(sigma2)
    epsilon = np.random.normal(0, sigma, n).reshape(n,1)
    X = np.random.multivariate_normal(np.zeros(p), sigma_rho, n)
    Y = X @ beta + epsilon
    return X, Y

In [19]:
X, Y = generate_data(100, 10, 0, 'sparse')
X.shape, Y.shape

((100, 10), (100, 1))